In [10]:
import urllib.request
import tarfile
import os

# URL of the dataset
url = "https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz"
dataset_folder = "rt-polaritydata"

# Function to download the dataset
def download_dataset(url, download_path):
    if not os.path.exists(download_path):
        print(f"Downloading dataset from {url}...")
        urllib.request.urlretrieve(url, download_path)
        print("Download complete.")
    else:
        print("Dataset already downloaded.")

# Function to extract the tar.gz file
def extract_dataset(tar_path, extract_to):
    if not os.path.exists(extract_to):
        print(f"Extracting {tar_path}...")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=extract_to)
        print(f"Extraction complete. Files extracted to {extract_to}")
    else:
        print("Dataset already extracted.")

# Download and extract dataset
download_path = "rt-polaritydata.tar.gz"
download_dataset(url, download_path)
extract_dataset(download_path, dataset_folder)

# List the extracted files
print(f"Extracted files: {os.listdir(dataset_folder)}")


Download complete.
Extracting rt-polaritydata.tar.gz...
Extraction complete. Files extracted to rt-polaritydata
Extracted files: ['rt-polaritydata.README.1.0.txt', 'rt-polaritydata']


In [11]:
%cd rt-polaritydata/rt-polaritydata

/content/rt-polaritydata/rt-polaritydata/rt-polaritydata/rt-polaritydata


In [12]:
%ls

rt-polarity.neg  rt-polarity.pos


In [13]:
# In Colab, install TensorFlow Hub and TensorFlow
!pip install tensorflow tensorflow-hub


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle

# 1. Load the data from files
def load_data(positive_file, negative_file):
    # Reading positive and negative data
    with open(positive_file, 'r', encoding='latin-1') as pos_file:
        positive_data = pos_file.readlines()

    with open(negative_file, 'r', encoding='latin-1') as neg_file:
        negative_data = neg_file.readlines()

    # Strip leading/trailing whitespaces
    positive_data = [line.strip() for line in positive_data]
    negative_data = [line.strip() for line in negative_data]

    return positive_data, negative_data

# 2. Label the data
def prepare_dataframe(positive_data, negative_data):
    # Create dataframes for positive and negative data
    pos_df = pd.DataFrame({'text': positive_data, 'label': 1})  # 1 for positive
    neg_df = pd.DataFrame({'text': negative_data, 'label': 0})  # 0 for negative

    # Concatenate both dataframes
    data = pd.concat([pos_df, neg_df], ignore_index=True)

    # Shuffle the dataset
    data = shuffle(data).reset_index(drop=True)

    return data

# 3. Split data into training, validation, and test sets
def split_data(data):
    # Split into training (4000 each), validation (500 each), and test sets (831 each)
    train_pos = data[data['label'] == 1][:4000]
    train_neg = data[data['label'] == 0][:4000]
    val_pos = data[data['label'] == 1][4000:4500]
    val_neg = data[data['label'] == 0][4000:4500]
    test_pos = data[data['label'] == 1][4500:]
    test_neg = data[data['label'] == 0][4500:]

    # Combine positive and negative for each set
    train_data = pd.concat([train_pos, train_neg], ignore_index=True)
    val_data = pd.concat([val_pos, val_neg], ignore_index=True)
    test_data = pd.concat([test_pos, test_neg], ignore_index=True)

    # Shuffle each set to mix positives and negatives
    train_data = shuffle(train_data).reset_index(drop=True)
    val_data = shuffle(val_data).reset_index(drop=True)
    test_data = shuffle(test_data).reset_index(drop=True)

    return train_data, val_data, test_data

# 4. Vectorize the text using TF-IDF
def vectorize_data(train_data, val_data, test_data):
    # Use TF-IDF vectorizer to convert text to numerical features
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

    # Fit on training data and transform
    X_train = vectorizer.fit_transform(train_data['text'])
    X_val = vectorizer.transform(val_data['text'])
    X_test = vectorizer.transform(test_data['text'])

    # Extract labels
    y_train = train_data['label']
    y_val = val_data['label']
    y_test = test_data['label']

    return X_train, X_val, X_test, y_train, y_val, y_test


# Main function to run preprocessing
def preprocess_data(positive_file, negative_file):
    # Load the data
    positive_data, negative_data = load_data(positive_file, negative_file)

    # Prepare and shuffle the dataframe
    data = prepare_dataframe(positive_data, negative_data)

    # Split the data into training, validation, and test sets
    train_data, val_data, test_data = split_data(data)

    # Vectorize the data using TF-IDF
    # X_train, X_val, X_test, y_train, y_val, y_test = vectorize_data(train_data, val_data, test_data)

    # return X_train, X_val, X_test, y_train, y_val, y_test
    return train_data, val_data, test_data

# Paths to the data files
positive_file = 'rt-polarity.pos'
negative_file = 'rt-polarity.neg'

# Preprocess the data
train_data, val_data, test_data = preprocess_data(positive_file, negative_file)

# You can now use X_train, X_val, X_test, y_train, y_val, y_test for model training and evaluation.


In [14]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional


In [47]:
# Load the ELMo model from TensorFlow Hub
elmo = hub.load("https://tfhub.dev/google/elmo/3")

# Function to get ELMo embeddings
# def elmo_embedding(text):
#     embeddings = elmo.signatures["default"](tf.constant([text]))["elmo"]
#     return tf.reduce_mean(embeddings, axis=1).numpy()

# # Test embedding for one sentence
# print(elmo_embedding("This movie was absolutely amazing!"))

def elmo_embedding(sentences):
    # ELMo expects a list of sentences
    embeddings = elmo.signatures['default'](tf.convert_to_tensor(sentences))['elmo']
    # Averaging the embeddings for each sentence
    # return np.array([np.mean(embedding, axis=0) for embedding in embeddings])
    return np.array(embeddings)

sample_sentences = ["This is a great movie.", "I didn't enjoy this film."]
elmo_embeddings = elmo_embedding(sample_sentences)
print(elmo_embeddings.shape)

(2, 5, 1024)


In [48]:
# def convert_to_elmo_embeddings(data):
#     embeddings = [elmo_embedding(sentence) for sentence in data['text']]
#     return np.vstack(embeddings)

def convert_to_elmo_embeddings(data):
    embeddings = elmo_embedding(data['text'].tolist())
    return embeddings



# Convert the datasets
X_train = convert_to_elmo_embeddings(train_data)
X_val = convert_to_elmo_embeddings(val_data)
X_test = convert_to_elmo_embeddings(test_data)

# Extract labels
y_train = train_data['label'].values
y_val = val_data['label'].values
y_test = test_data['label'].values

# Check shape of embeddings
print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'X_test shape: {X_test.shape}')


X_train shape: (8000, 59, 1024)
X_val shape: (1000, 55, 1024)
X_test shape: (1662, 54, 1024)


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


# Model training
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Validation
val_preds = clf.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, val_preds)}")

# Testing
test_preds = clf.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, test_preds)}")
print(classification_report(y_test, test_preds))


Validation Accuracy: 0.791
Test Accuracy: 0.8074608904933814
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       831
           1       0.81      0.80      0.81       831

    accuracy                           0.81      1662
   macro avg       0.81      0.81      0.81      1662
weighted avg       0.81      0.81      0.81      1662



In [24]:
from sklearn import svm
print("\nTraining SVM Classifier...")
svm_clf = svm.SVC(kernel='linear', C=1.0)
svm_clf.fit(X_train, y_train)

# Validation accuracy for SVM
val_preds_svm = svm_clf.predict(X_val)
print(f"SVM Validation Accuracy: {accuracy_score(y_val, val_preds_svm)}")

# Test accuracy for SVM
test_preds_svm = svm_clf.predict(X_test)
print(f"SVM Test Accuracy: {accuracy_score(y_test, test_preds_svm)}")
print("SVM Classification Report:")
print(classification_report(y_test, test_preds_svm))



Training SVM Classifier...
SVM Validation Accuracy: 0.787
SVM Test Accuracy: 0.8038507821901324
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.82      0.81       831
           1       0.82      0.79      0.80       831

    accuracy                           0.80      1662
   macro avg       0.80      0.80      0.80      1662
weighted avg       0.80      0.80      0.80      1662



In [50]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Dropout
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Build BiLSTM Model
def build_bilstm_model(input_shape):
    model = Sequential()

    # Add a bidirectional LSTM layer
    model.add(Bidirectional(LSTM(units=128, return_sequences=False), input_shape=input_shape))

    # Add a dropout layer to avoid overfitting
    model.add(Dropout(0.5))

    # Add a dense layer for output
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# 2. Train the BiLSTM model
def train_bilstm_model(X_train, y_train, X_val, y_val):


    input_shape = (None, X_train.shape[2])  # Shape: (sequence_length, embedding_dimension)

    # Build the BiLSTM model
    model = build_bilstm_model(input_shape)

    # Print the model summary
    model.summary()

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=5,          # You can tune the number of epochs
        batch_size=32,     # Adjust batch size based on available memory
        verbose=1
    )

    return model, history

# 3. Evaluate the model on test data
def evaluate_model(model, X_test, y_test):
    # Predict on test data
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary (0 or 1)

    # Evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    # Classification report and confusion matrix
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# Example Usage
# Assuming you've already generated ELMo embeddings as X_train, X_val, X_test
# And your labels are stored in y_train, y_val, y_test

# Train the model
bilstm_model, history = train_bilstm_model(X_train, y_train, X_val, y_val)

# Evaluate the model on the test set
evaluate_model(bilstm_model, X_test, y_test)


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_10 (Bidirect  (None, 256)               1180672   
 ional)                                                          
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 1)                 257       
                                                                 
Total params: 1180929 (4.50 MB)
Trainable params: 1180929 (4.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
250/250 [==============================] - 24s 85ms/step - loss: 0.5164 - accuracy: 0.7408 - val_loss: 0.4678 - val_accuracy: 0.7800
Epoch 2/5
250/250 [==============================] - 21s 82ms/step - l